## testes de IA

In [1]:
# Instalar pacotes necessários para a API do Gemini e para carregar o arquivo .env (Ignorar o break-system-packages, eu que precisei fazer isso)
!pip install -q google-generativeai python-dotenv --break-system-packages

In [7]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import pathlib
import ipywidgets as widgets
from IPython.display import display, clear_output
import tempfile

In [ ]:
env_path = pathlib.Path.cwd() / ".env"
print(f"Diretório atual: {pathlib.Path.cwd()}")
print(f"Arquivo chave.env existe? {env_path.exists()}")

if env_path.exists():
    load_dotenv(dotenv_path=str(env_path))
else:
    raise FileNotFoundError(f"Crie {env_path} com: API_KEY=sua_chave")

# 2. Ler API_KEY
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    if env_path.exists():
        print("\nConteúdo do chave.env:")
        print(env_path.read_text())
    raise ValueError("Verifique API_KEY no chave.env (sem espaços/extras).")

print(f"Chave carregada: {API_KEY[:8]}...")

genai.configure(api_key=API_KEY)

print("\nModelos disponíveis para generateContent (com suporte a PDFs):")
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods and "gemini" in m.name:
        print(f"  - {m.name} ({m.display_name})")

Diretório atual: /home/jordao/Documentos/Projeto_IA
Arquivo chave.env existe? True
Chave carregada: AIzaSyD6...

Modelos disponíveis para generateContent (com suporte a PDFs):
  - models/gemini-2.5-pro-preview-03-25 (Gemini 2.5 Pro Preview 03-25)
  - models/gemini-2.5-flash-preview-05-20 (Gemini 2.5 Flash Preview 05-20)
  - models/gemini-2.5-flash (Gemini 2.5 Flash)
  - models/gemini-2.5-flash-lite-preview-06-17 (Gemini 2.5 Flash-Lite Preview 06-17)
  - models/gemini-2.5-pro-preview-05-06 (Gemini 2.5 Pro Preview 05-06)
  - models/gemini-2.5-pro-preview-06-05 (Gemini 2.5 Pro Preview)
  - models/gemini-2.5-pro (Gemini 2.5 Pro)
  - models/gemini-2.0-flash-exp (Gemini 2.0 Flash Experimental)
  - models/gemini-2.0-flash (Gemini 2.0 Flash)
  - models/gemini-2.0-flash-001 (Gemini 2.0 Flash 001)
  - models/gemini-2.0-flash-exp-image-generation (Gemini 2.0 Flash (Image Generation) Experimental)
  - models/gemini-2.0-flash-lite-001 (Gemini 2.0 Flash-Lite 001)
  - models/gemini-2.0-flash-lite (Ge

In [9]:
def query_pdf(pdf_path, question, model_name='models/gemini-2.5-flash'):
    """
    Faz upload de um PDF para o Gemini e responde a uma pergunta.
    
    Args:
    - pdf_path: Caminho para o PDF
    - question: Sua pergunta
    - model_name: Use 'models/gemini-2.5-flash' (rápido) ou 'models/gemini-2.5-pro' (avançado)
    
    Nota: Modelos 2.x suportam PDFs até 2GB/300 páginas.
    """
    uploaded_file = genai.upload_file(path=pdf_path)
    model = genai.GenerativeModel(model_name)
    response = model.generate_content([uploaded_file, question])
    print("Resposta do Gemini:")
    print(response.text)
    
    return response.text

In [10]:
uploader = widgets.FileUpload(
    accept='.pdf', multiple=False, description='Selecionar PDF'
)

question_input = widgets.Textarea(
    value='', placeholder='Digite sua pergunta sobre o PDF...',
    description='Pergunta:', layout=widgets.Layout(width='100%', height='100px')
)

run_button = widgets.Button(
    description='Enviar Pergunta', button_style='success',
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

def on_run_button_clicked(b):
    with output_area:
        clear_output()
        if not uploader.value:
            print("Erro: Selecione um PDF.")
            return
        if not question_input.value.strip():
            print("Erro: Digite uma pergunta.")
            return

        uploaded_file_info = uploader.value[0]
        pdf_name = uploaded_file_info.name
        pdf_bytes = uploaded_file_info.content.tobytes()

        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            tmp_file.write(pdf_bytes)
            temp_path = tmp_file.name

        print(f"PDF: {pdf_name}")
        print(f"Pergunta: {question_input.value}\n")
        print("Processando...")

        try:
            resposta = query_pdf(temp_path, question_input.value)
        except Exception as e:
            print(f"Erro: {e}")
        finally:
            if os.path.exists(temp_path):
                os.unlink(temp_path)

run_button.on_click(on_run_button_clicked)
display(uploader, question_input, run_button, output_area)

FileUpload(value=(), accept='.pdf', description='Selecionar PDF')

Textarea(value='', description='Pergunta:', layout=Layout(height='100px', width='100%'), placeholder='Digite s…

Button(button_style='success', description='Enviar Pergunta', layout=Layout(width='200px'), style=ButtonStyle(…

Output()